In [70]:
#0 Set anything that needs to be changed at the beginning:  $$$ LabelPath and GraphArtifactsFolderName
# You have to restart the kernel each time it seems or sometimes Cell-22 spits an  error!!
# Better clear all output too while you're at it!
# Let's call this: VER2.1, Version-2.1: orders-4, output*0.38mm
# Cases Done: [71, 2, 37, 18, 16, 88, 19, 90, 11, 6, 46, 39, 22, 21, 95, 84, 50,]

# Only need to change here: Look for S-Final---label.nrrd:
LabelPath = "y:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-46-Spiromics-51186800/Markups/88-Jensen/S-Final-2-label.nrrd"
GraphArtifactsFolderName = "./GraphOutputSoheil"
verbose = False # Set to True to see a lot more print outputs

In [71]:
#1 Load/create the masks: avmlabel, atree

import numpy as np
import nrrd  # pip install pynrrd
import re

# CasePath is everything before the final "/"
# Find the last occurrence of '/'
last_slash_index = LabelPath.rfind('/')
# Slice the string up to the last '/'
casepath = LabelPath[:last_slash_index + 1] # .../1-Done/Case-56-Spiromics-48876393/Markups/88-Nate/

avmlabel, metadata = nrrd.read(LabelPath)
## Extract Case Number
pattern = r"Case-(\d+)-"

# Use re.findall() to find all matches in the string
matches = re.findall(pattern, casepath)

# Extract the first matched number (if any)
if matches:
    CaseNumber = int(matches[0])
    print("Extracted Case Number:", CaseNumber) # To be used numerous times in the code
else:
    print("No 'Case-' followed by a number found in the string.")

atree = np.zeros_like(avmlabel)
atree[avmlabel == 1] = 1

Extracted Case Number: 46


In [72]:
#2 Inspect using mayavi: Check saved A-tree snapshot

import winsound
from mayavi import mlab
import os

frequency = 1000  # Frequency of the beep in Hertz
duration = 1000  # Duration of the beep in milliseconds
# winsound.Beep(frequency, duration)

figure = mlab.figure(bgcolor=(1, 1, 1))
contour = mlab.contour3d(atree, contours=[0.5], color=(1, 0, 0))

if not os.path.exists(GraphArtifactsFolderName):
    os.mkdir(GraphArtifactsFolderName)

mlab.savefig(GraphArtifactsFolderName + f"/AtreeSnapshot-{CaseNumber}.png")  # You can specify the filename and format
mlab.close()

In [73]:
#3 Create/Fetch Skeleton

from skimage.morphology import skeletonize_3d  # pip install scikit-image
import numpy as np
import nrrd  # pip install pynrrd

# SkeletonPathName  # Formerly f"./Data/Artifacts/E-Case-{CaseNumber}-A-Skeleton.nrrd"
# SkeletonPathName = casepath + "S-Final-2-label-Skeleton.nrrd"
SkeletonPathName = GraphArtifactsFolderName + f"/K-Case-{CaseNumber}-Final2Label-Skeleton.nrrd"

# Create and save the skeleton:
# skeleton = skeletonize_3d(atree.astype(np.uint8))
# nrrd.write(SkeletonPathName, skeleton)

# Or read it in:
skeleton, _ = nrrd.read(SkeletonPathName)

In [74]:
#4 Compute/Fetch Distance Transform
# Contains the Euclidean distance from each voxel to the nearest background voxel

from scipy.ndimage import distance_transform_edt

# EdtPathName = f"./Data/Artifacts/I-Case-{CaseNumber}-EDT.nrrd"
# EdtPathName = casepath + f"I-Case-{CaseNumber}-EDT.nrrd"
EdtPathName = GraphArtifactsFolderName + f"/T-Case-{CaseNumber}-EDT.nrrd"
# Compute the distance transform
# distance_transform = distance_transform_edt(atree)
# nrrd.write(EdtPathName, distance_transform)

# Or read it in
distance_transform, _ = nrrd.read(EdtPathName)

In [75]:
#5 Create Graph/Edges/Neighbours/...
# Step 2: Convert skeletonized image to a graph representation ########### Very bad code/ hard to understand *

import networkx as nx

graph = nx.Graph()
indices = np.array(np.where(skeleton)) # 3x1000?
num_points = indices.shape[1]
for i in range(num_points): # 20,100 # num_points
    point = tuple(indices[:, i])
    neighbors = []
    for offset in np.ndindex((3, 3, 3)): # (0,1,2 .. 0,1,2 .. 0,1,2)
        neighbor = tuple(indices[:, i] + (np.array(offset) - 1)) # >> -1,0,1
        if neighbor != point and neighbor in graph:
            neighbors.append(neighbor)
    graph.add_node(point)
    graph.add_edges_from((point, n) for n in neighbors)

In [76]:
#6 Print_Stats [Nodes/Edges/...]

def print_stats(graph):
    num_nodes = nx.number_of_nodes(graph)
    num_edges = nx.number_of_edges(graph)
    # all_nodes = list(graph.nodes)
    # all_edges = list(graph.edges)

    print("Number of Nodes:", num_nodes)
    print("Number of Edges:", num_edges)
    # print("All Nodes:", all_nodes)
    # print("All Edges:", all_edges)
    return

print_stats(graph)

Number of Nodes: 8130
Number of Edges: 8162


In [77]:
#7 Remove degree-2 nodes: Changes the graph *

def RemoveAndReconnectDegree2Nodes(graph):
    while True:
        # Find nodes with degree 2
        degree_two_nodes = [node for node, degree in graph.degree if degree == 2]
        
        # If there are no intermediate nodes to remove, break the loop
        if not degree_two_nodes:
            break    
        # Remove the intermediate nodes
        for node in degree_two_nodes:
            neighbors = list(graph.neighbors(node))
            if len(neighbors) == 2: ####### Redundant condition! *
                graph.add_edge(neighbors[0], neighbors[1])
            graph.remove_node(node)

RemoveAndReconnectDegree2Nodes(graph)

In [78]:
#8 print_stats
print_stats(graph)

Number of Nodes: 306
Number of Edges: 338


In [79]:
#9 Create/update 'edge_length' property for all edges *

def UpdateEdgeLengths(G):
    for edge in G.edges:
        node1, node2 = edge
        x1, y1, z1 = node1
        x2, y2, z2 = node2
        edge_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
        G.edges[edge]['edge_length'] = edge_length

UpdateEdgeLengths(graph)

In [80]:
#10 Function: Plot-1: plot_3D: Plot Interactive 3D *

import plotly.graph_objects as go  # pip install plotly # pip install nbformat
from plotly.subplots import make_subplots

def plot_3D(graph):
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
    x, y, z = zip(*graph.nodes)
    
    leaf_nodes = [node for node, degree in graph.degree if degree == 1]
    
    node_trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=2,
            color='red', #['red' if node in leaf_nodes else 'blue' for node in graph.nodes()],
            opacity=0.8
        )
    )
    fig.add_trace(node_trace)

    for edge in graph.edges():
        x0, y0, z0 = edge[0]
        x1, y1, z1 = edge[1]
        
        edge_length = graph.edges[edge]['edge_length']
        edge_text = f'{edge_length:.2f}'  # Display the edge length as text
        
        edge_trace = go.Scatter3d(
            x=[x0, x1],
            y=[y0, y1],
            z=[z0, z1],
            mode='lines',
            # line=dict(color='orange' if graph.edges[edge]['edge_length'] < 20 else 'gray',
            #           width=3),
            line=dict(color='gray', width=3),
            hoverinfo='text',
            text=edge_text
        )
        fig.add_trace(edge_trace)

    fig.update_layout(  
        scene=dict(
        # Set background color to transparent or any color you prefer
        bgcolor='rgba(0,0,0,0)',
        
        # Hide axes
        xaxis=dict(showbackground=False, showgrid=False, zeroline=False, showline=False),
        yaxis=dict(showbackground=False, showgrid=False, zeroline=False, showline=False),
        zaxis=dict(showbackground=False, showgrid=False, zeroline=False, showline=False)
        )
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis=dict(range=[0, max(x) + 1]),
            yaxis=dict(range=[0, max(y) + 1]),
            zaxis=dict(range=[0, max(z) + 1])
        )
    )

    fig.show()
    return

In [81]:
# ##11 Timed calling of plot_3D: Will take time to show up even when indicated done:

# import time
# StartTime = time.time()
# print('Plotting the 3D Graph ..')
# plot_3D(graph)
# print(f'Elapsed: {time.time()-StartTime:5.2f}')

In [82]:
#12 Find all 3-cycels: Approach-9 with no duplicates

# Function to find and print all length-3 cycles without duplicates
def ReFindAndPrintLength3Cycles(G, Prnt=False):

    length_3_cycles = set()
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        for neighbor1 in neighbors:
            for neighbor2 in neighbors:
                if neighbor1 != neighbor2 and G.has_edge(neighbor1, neighbor2):
                    cycle = tuple(sorted([node, neighbor1, neighbor2]))  # Sorting to avoid permutations
                    length_3_cycles.add(cycle)

    # Convert the cycles back to lists for printing
    length_3_cycles = [list(cycle) for cycle in length_3_cycles]

    # Print the length-3 cycles without duplicates
    if Prnt:
        print(len(length_3_cycles), 'simple+complex 3-cycles exist\n')
        # for cycle in length_3_cycles:
        #     print(cycle)
    return length_3_cycles

length_3_cycles = ReFindAndPrintLength3Cycles(graph, Prnt=True)

33 simple+complex 3-cycles exist



In [83]:
#13* Keep an intact copy of the graph:

import copy
graphintact = copy.deepcopy(graph)
print("An intact copy of the graph kept in the variable 'graphintact'. Use it when things are messed up.")

An intact copy of the graph kept in the variable 'graphintact'. Use it when things are messed up.


In [84]:
#RR To refresh your graph once it's messed below, come back to this:
# graph = copy.deepcopy(graphintact)

In [85]:
#14 Only once: This changes the graph: Resolve isolated 3-cycles by merging the 3 nodes and reconnecting edges

print_stats(graph)
print()
length_3_cycles = ReFindAndPrintLength3Cycles(graph, Prnt=False)

for cycle in length_3_cycles:

    # For each cycle, find the outer/final neighbors
    finalneighbors = []
    for node in cycle:
        DEGR = graph.degree(node)
        if DEGR != 3:
            # print(Fore.YELLOW + f'\nThis is not a simple cycle. Node {node} is degree {DEGR}. Breaking ..' + Style.RESET_ALL)
            if verbose: print(f'{cycle} is not a simple cycle. Node {node} is degree {DEGR}.')
            break
        finalneighbors.extend([neighbor for neighbor in graph.neighbors(node) if neighbor not in cycle])
    if verbose:
        if DEGR == 3:
            print('cycle:', cycle)
    # print('finalneighbors:', finalneighbors)
    if DEGR != 3:
        print("Skipping to the next cycle ..\n")
        continue

    # Takes the 3 nodes in the cycle and forms their average
    NewAverageNode = tuple(round((a + b + c) / 3) for a, b, c in zip(cycle[0], cycle[1], cycle[2])) # zip(tuple1, tuple2, tuple3)
    # print('NewAverageNode:', NewAverageNode)

    # If NewAverageNode not on the skeleton, go to its neighborhood and find the nearest voxel that lies on the skeleton:
    if skeleton[*NewAverageNode] == 0:
        print(f"NewAverageNode {NewAverageNode} not on the skeleton.")
        xxna, yyna, zzna = NewAverageNode
        success = False
        NeighbrhdSize = 3
        while success == False:
            print(f"Searching neighborhood {NeighbrhdSize}")
            lowerbnd = -(NeighbrhdSize//2)
            upperbnd =  (NeighbrhdSize//2) + 1
            neighbors = [(i, j, k) for i in range(lowerbnd, upperbnd) for j in range(lowerbnd, upperbnd) for k in range(lowerbnd, upperbnd)]
            neighbors.remove((0,0,0))
            for neighbor_offset in neighbors:
                nnx, nny, nnz = xxna + neighbor_offset[0], yyna + neighbor_offset[1], zzna + neighbor_offset[2]
                if 0 <= nnx < skeleton.shape[0] and 0 <= nny < skeleton.shape[1] and 0 <= nnz < skeleton.shape[2]:
                    if skeleton[nnx, nny, nnz] != 0:
                        NewAverageNode = (nnx, nny, nnz)
                        print(f"Adjusted the NewAverageNode to {NewAverageNode}")
                        success = True
                        break            
            NeighbrhdSize += 2

    # Now remove the old 3 nodes in cycle
    for node in cycle:
        graph.remove_node(node)
    # print('Removed the 3 nodes in cycle')

    # Add the new average node
    graph.add_node(NewAverageNode)
    # print('Added new average node')

    # Make the new edges/connections
    for node in finalneighbors:
        graph.add_edge(node, NewAverageNode)
    # print('Added the final edges')

    print('Resolved the 3-cycle.')
    UpdateEdgeLengths(graph)
    # print_stats(graph)
    if verbose: print()

print()
print_stats(graph)

Number of Nodes: 306
Number of Edges: 338

NewAverageNode (240, 347, 442) not on the skeleton.
Searching neighborhood 3
Adjusted the NewAverageNode to (240, 347, 443)
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
NewAverageNode (242, 337, 293) not on the skeleton.
Searching neighborhood 3
Adjusted the NewAverageNode to (241, 337, 293)
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
NewAverageNode (572, 308, 291) not on the skeleton.
Searching neighborhood 3
Adjusted the NewAverageNode to (572, 308, 290)
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
NewAverageNode (435, 255, 382) not on the skeleton.
Searching neighborhood 3
Adjusted the NewAverageNode to (434, 255, 382)
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.
Resolved the 3-cycle.

In [86]:
#15 Create updated list of all edges in cycles: [they're probably complex cycles now at this point]

length_3_cycles = ReFindAndPrintLength3Cycles(graph)
print(f"{len(length_3_cycles)} complex 3-cycles exist")

EdgesInCyclesWithDuplicates = [[L[i], L[(i+1)%3]] for L in length_3_cycles for i in [0, 1, 2]]

EdgesInCycles = []
for edge in EdgesInCyclesWithDuplicates:
    if ([edge[0],edge[1]] not in EdgesInCycles) and ([edge[1],edge[0]] not in EdgesInCycles):
         EdgesInCycles.append(edge)
        #  print(f'Appended {edge} to EdgesInCycles list')

print(len(EdgesInCycles), 'edges in complex 3-cycles exist')

0 complex 3-cycles exist
0 edges in complex 3-cycles exist


In [87]:
#16 Plot-2 Interactive 3D-2!: Check #Clusters: Show Cycles in Red >> Edges: {Red=InCycle, Grey=Else}, Nodes: {Grey}#{Red=Leaf, Blue=Else}

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
x, y, z = zip(*graph.nodes)

leaf_nodes = [node for node, degree in graph.degree if degree == 1]

node_trace = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color='grey', #['red' if node in leaf_nodes else 'blue' for node in graph.nodes()], # 'grey',
        opacity=0.1 #0.8
    )
)
fig.add_trace(node_trace)

for edge in graph.edges():
    x0, y0, z0 = edge[0]
    x1, y1, z1 = edge[1]
    
    edge_length = graph.edges[edge]['edge_length']
    edge_text = f'{edge_length:.2f}'  # Display the edge length as text
    
    edge_is_in_a_cycle = [edge[0],edge[1]] in EdgesInCycles or [edge[1],edge[0]] in EdgesInCycles
    
    edge_trace = go.Scatter3d(
        x=[x0, x1],
        y=[y0, y1],
        z=[z0, z1],
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 0.8)' if edge_is_in_a_cycle else 'rgba(128, 128, 128, 0.4)',
            width=8 if edge_is_in_a_cycle else 1),
        hoverinfo='text',
        text=edge_text
    )
    fig.add_trace(edge_trace)

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(range=[0, max(x) + 1]),
        yaxis=dict(range=[0, max(y) + 1]),
        zaxis=dict(range=[0, max(z) + 1])
    )
)

fig.show()

In [88]:
#17 Create the nodes list of the complex cycle: Works only for one complex cluster!

CxNodesList = [] # Complex nodes list
NodeIndex = 0
for edge in EdgesInCycles: ############### EdgesInCycles should be adjusted for other scenarios! ***
    for i in [0,1]:
        CurrentNode = edge[i]
        if CurrentNode not in CxNodesList:
            CxNodesList.append(CurrentNode)
            print(f'Node[{NodeIndex}]: Appended {CurrentNode} to CxNodesList')
            NodeIndex += 1
            
print(f'\nOverall {len(CxNodesList)} nodes in CxNodesList')


Overall 0 nodes in CxNodesList


In [89]:
#18 Specify n_clusters and get the clusters as separate lists $$$ n_clusters (judge visually and from code block above)
# NO NEED TO RUN CODE BLOCK 18-20 IF 0 NODES IN "CxNodesList"

if CxNodesList: # If not empty, do ..

    from sklearn.cluster import KMeans # pip install scikit-learn

    # Convert the list of 3-tuples to a NumPy array for K-Means clustering
    data = np.array(CxNodesList)

    # Specify the number of clusters (in this case, 3 clusters)
    # n_clusters = 1
    winsound.Beep(frequency, duration)
    n_clusters = int(input("Please enter the number of clusters: [Cell-18>>Cell-16]: "))

    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)  # You can set n_init to any desired value
    kmeans.fit(data)

    # Get the cluster labels for each data point
    cluster_labels = kmeans.labels_

    # Initialize empty lists for each cluster
    clusters = [[] for _ in range(n_clusters)]

    # Organize data points into clusters based on labels
    for i, label in enumerate(cluster_labels):
        clusters[label].append(CxNodesList[i])

    # Print the clusters
    for i, cluster in enumerate(clusters):
        print(f"Cluster {i + 1}:", cluster)

In [90]:
#19 Function: Resolves the complex cycles for one cluster: Changes the graph! Works only once!

def ResolveComplexCluster(CxNodesCluster, graph):

    # List the neighbors that lie outside the cx-cycle
    OuterNeighbors = []
    print("Outer Neighbors:")
    for node in CxNodesCluster:
        for neighbor in graph.neighbors(node):
            if neighbor not in CxNodesCluster and neighbor not in OuterNeighbors:
                OuterNeighbors.append(neighbor)
                print(neighbor)

    # Print the cx-cycle
    print("\nCx-Cycle:")
    for node in CxNodesCluster:
        print(node)

    # Takes the Cx-Nodes and forms their average
    # CxNodesList: List of n 3-tuples  # n: Number of Cx Nodes  # After zip becomes 3 x ntuple
    NewAverageNode = tuple([round(sum(ntuple)/len(ntuple)) for ntuple in zip(*CxNodesCluster)])
    print('\nNewAverageNode:')
    print(NewAverageNode)

    # If NewAverageNode not on the skeleton, go to its neighborhood and find the nearest voxel that lies on the skeleton:
    if skeleton[*NewAverageNode] == 0:
        print(f"NewAverageNode {NewAverageNode} not on the skeleton.")
        xxna, yyna, zzna = NewAverageNode
        success = False
        NeighbrhdSize = 3
        while success == False:
            print(f"Searching neighborhood {NeighbrhdSize}")
            lowerbnd = -(NeighbrhdSize//2)
            upperbnd =  (NeighbrhdSize//2) + 1
            neighbors = [(i, j, k) for i in range(lowerbnd, upperbnd) for j in range(lowerbnd, upperbnd) for k in range(lowerbnd, upperbnd)]
            neighbors.remove((0,0,0))
            for neighbor_offset in neighbors:
                nnx, nny, nnz = xxna + neighbor_offset[0], yyna + neighbor_offset[1], zzna + neighbor_offset[2]
                if 0 <= nnx < skeleton.shape[0] and 0 <= nny < skeleton.shape[1] and 0 <= nnz < skeleton.shape[2]:
                    if skeleton[nnx, nny, nnz] != 0:
                        NewAverageNode = (nnx, nny, nnz)
                        print(f"Adjusted the NewAverageNode to {NewAverageNode}")
                        success = True
                        break            
            NeighbrhdSize += 2

    # Now remove the old nodes in cx-cycle
    for node in CxNodesCluster:
        graph.remove_node(node)
    print(f'\nRemoved the {len(CxNodesCluster)} nodes in cx-cycle')

    # Add the new average node
    graph.add_node(NewAverageNode)
    print('Added new average node')

    # Make the new edges/connections
    for node in OuterNeighbors:
        graph.add_edge(node, NewAverageNode)
    print('Added the final edges\n')

    UpdateEdgeLengths(graph)
    print_stats(graph)
    print()

    ReFindAndPrintLength3Cycles(graph)

In [91]:
#20 Execute: Resolve all clusters:

if CxNodesList: # If not empty, do ..
    for cluster in clusters:
        ResolveComplexCluster(cluster, graph)

In [92]:
#21 Check if it could be a tree:

def CheckBeingATree():

    num_nodes = nx.number_of_nodes(graph)
    num_edges = nx.number_of_edges(graph)

    print("Number of Nodes:", num_nodes)
    print("Number of Edges:", num_edges)

    if num_nodes != num_edges + 1: print("\nThis is not a tree: #nodes != #edges + 1")
    if num_nodes == num_edges + 1: print("\nThis could be a tree.")

CheckBeingATree()

Number of Nodes: 240
Number of Edges: 239

This could be a tree.


In [93]:
#21-1 To simplify tree further: Set and visualize SmallInsideEdges, to be removed: Displayed red:

UpdateEdgeLengths(graph)

# Case-79 root: (480, 196 , 396)
# leaf_nodes = [node for node, degree in graph.degree if degree == 1]

# size=[6 if node in leaf_nodes else 2 for node in graph.nodes()],
# color=['black' if node in leaf_nodes else 'grey' for node in graph.nodes()], # 'grey',
# edge_length = graph.edges[edge]['edge_length']
# SpecialCondition = edge_length < 10
# line=dict(color='rgba(255, 0, 0, 0.6)' if SpecialCondition else 'rgba(128, 128, 128, 0.6)',
#             width=6 if SpecialCondition else 2),

fig = mlab.figure(size=(2500, 1400), bgcolor=(1, 1, 1))

x, y, z = zip(*graph.nodes)
mlab.points3d(x, y, z, scale_factor=2, color=(0.2, 0.2, 0.2), opacity=0.7) # color=(1, 0, 0)

# Find all terminal edges smaller than LengthThreshold and deeper than InsideMargin
InsideMargin = 14
LengthThreshold = 14
SmallInsideEdges = [edge for edge in graph.edges if
                   (graph.degree(edge[0]) == 1 or graph.degree(edge[1]) == 1) and
                   (graph.edges[edge]['edge_length'] < LengthThreshold) and

                   ((graph.degree(edge[0]) == 1 and distance_transform[*edge[0]]>InsideMargin) or
                    (graph.degree(edge[1]) == 1 and distance_transform[*edge[1]]>InsideMargin))
                ]

print(f"How many small edges inside: {len(SmallInsideEdges)}")

# Create lines for edges
for edge in graph.edges:
    src, tgt = edge
    x = [src[0], tgt[0]]
    y = [src[1], tgt[1]]
    z = [src[2], tgt[2]]
    mlab.plot3d(x, y, z, tube_radius=0.2,
                color=(1, 0, 0) if edge in SmallInsideEdges else (0.2, 0.2, 0.2),#(0.2, 0.2, 0.2),
                opacity=0.7)

# Create contour for the 3D object
mlab.contour3d(atree, contours=[0.5], opacity=0.2, color=(0, 151/255, 206/255))

winsound.Beep(frequency, duration)
mlab.show()

How many small edges inside: 12


In [94]:
#21-2 Keep another copy of the graph:

graphintact2 = copy.deepcopy(graph)
skeletonintact2 = copy.deepcopy(skeleton)

#RR To refresh your graph once it's messed below, come back to this:
# graph = copy.deepcopy(graphintact2)
# skeleton = copy.deepcopy(skeletonintact2)

In [95]:
#22 Function: Traverse edge and calculate average diameter function, given endpoint indices and skeleton *

import numpy as np
import copy
import random
import time

def traverse_edge_and_average(skeletonn, edt, index1, index2, EdgeNum): # edt: eucledean distance transform

    global verbose
    global StartTime
    
    # normalize_vector = lambda vec: vec / np.linalg.norm(vec)
    if verbose: print(f"Trying to go from {index1} to {index2}")

    # Define the neighborhood offsets for 3D traversal (26-connected)
    neighbors = [(i, j, k) for i in range(-1, 2) for j in range(-1, 2) for k in range(-1, 2)]
    neighbors.remove((0,0,0))
    
    average_diameter = None
    timestried = 0
    while average_diameter == None and timestried < 2000:

        # Initialize variables to store diameter values and count
        if verbose: print(f"Attempt {timestried+1}: ", end='')
        skeletonnn = copy.deepcopy(skeletonn)
        diameter_sum = 0.0
        count = 0

        # Start traversal at index1
        current_index = index1
        visitedvoxels = []

        while current_index != index2:

            # print("current_index:", current_index) # Great for debugging

            if current_index == None: break
            visitedvoxels.append(current_index)

            # Get the current voxel's coordinates
            x, y, z = current_index

            # Check if the current voxel is within the skeleton and within bounds
            if 0 <= x < skeletonnn.shape[0] and 0 <= y < skeletonnn.shape[1] and 0 <= z < skeletonnn.shape[2]:
                if skeletonnn[x, y, z] != 0:
                    # If the current voxel is part of the skeleton, add its diameter to the sum
                    diameter_sum += edt[x, y, z]
                    count += 1

                    # Mark the current voxel as visited (you can set it to 0 if necessary)
                    skeletonnn[x, y, z] = 0  # Uncomment this line if you want to mark visited voxels
                    
                    max_alignment = float('-inf')
                    next_index = None


                    ## Logic to choose next voxel:

                    # Logic-1: Deterministic:
                    # for neighbor_offset in neighbors:
                    #     nx, ny, nz = x + neighbor_offset[0], y + neighbor_offset[1], z + neighbor_offset[2]

                    #     strghtdirectiont = tuple(np.array(index2) - np.array(current_index))
                    #     strghtdirectionu = normalize_vector(strghtdirectiont)
                    #     neighbor_offsetu = normalize_vector(neighbor_offset )
                    #     currentalignment = np.dot(neighbor_offsetu, strghtdirectionu)
                        
                    #     if 0 <= nx < skeletonn.shape[0] and 0 <= ny < skeletonn.shape[1] and 0 <= nz < skeletonn.shape[2]:
                    #         if skeletonn[nx, ny, nz] != 0 and currentalignment > max_alignment:
                    #             max_alignment = currentalignment
                    #             next_index = (nx, ny, nz)

                    # Logic-2: Random walk:
                    while True:
                        if not any([skeletonnn[*tuple(np.array(current_index)+np.array(neigh))] for neigh in neighbors]): break
                        neighbor_offset = random.choice(neighbors)
                        nnx, nny, nnz = x + neighbor_offset[0], y + neighbor_offset[1], z + neighbor_offset[2]
                        if 0 <= nnx < skeletonnn.shape[0] and 0 <= nny < skeletonnn.shape[1] and 0 <= nnz < skeletonnn.shape[2]:
                            if skeletonnn[nnx, nny, nnz] != 0:
                                next_index = (nnx, nny, nnz)
                                break


                    # Update the current index to the next voxel in the skeleton
                    current_index = next_index
                else:
                    # If the current voxel is not part of the skeleton, stop the traversal
                    break
            else:
                # If the current index is out of bounds, stop the traversal
                break
        
        if current_index != index2:
            if verbose: print(f"Error! Couldn't get from {index1} to {index2}! Returned None.")
            pass

        # Set the output
        if count > 0 and current_index == index2:
            average_diameter = diameter_sum / count

        del skeletonnn
        timestried += 1

    if current_index == index2:
        if verbose: print(f"\nDone. Got to {index2}\n")
        pass

    num_edges = nx.number_of_edges(graph)

    try:
        # Your statement or code that might raise an error goes here
        print(f"Edge #{EdgeNum:3}/{num_edges}; AvgDiam: {average_diameter:010.7f}; TimeElapsed: {(time.time() - StartTime):07.2f}" + ("\n" if verbose else ""))
    except Exception as e:
        # This block will execute if an error occurs
        winsound.Beep(4000, 300)    
    # print(f"Edge #{EdgeNum:3}/{num_edges}; AvgDiam: {average_diameter:010.7f}; TimeElapsed: {(time.time() - StartTime):07.2f}" + ("\n" if verbose else ""))

    return average_diameter, visitedvoxels

In [96]:
#21-3 Given 'graph': Find and remove all edges in SmallInsideEdges: Changes the graph:
# All terminal edges smaller than LengthThreshold and deeper than InsideMargin

StartTime = time.time()

visitedvoxelstot = []

for indxe, edge in enumerate(SmallInsideEdges):

    node1, node2 = edge

    Node1IsOrWasDegree1 = graph.degree(node1)==1
    
    if graph.degree(node1)==1: graph.remove_node(node1)
    if graph.degree(node2)==1: graph.remove_node(node2)

    # TheDegree1Node = node1 if Node1IsOrWasDegree1 else node2
    # NodeOnMainstem = node2 if Node1IsOrWasDegree1 else node1

    # print(f"Trying to remove from skeleton: {TheDegree1Node}->{NodeOnMainstem}")

    # skeletonn = copy.deepcopy(skeleton)
    # avgdiam, visitedvoxels = traverse_edge_and_average(skeletonn, distance_transform, TheDegree1Node, NodeOnMainstem, 5)
    # visitedvoxelstot.append(visitedvoxels)
    # del skeletonn

    # print(indxe)
    # print("visitedvoxels:", visitedvoxels)
    # print()

# for vxt in visitedvoxelstot:
#     if len(vxt)>1:
#         for vx in vxt:
#             skeleton[*vx] = 0
#             print(f"deleted voxel: {vx}")

RemoveAndReconnectDegree2Nodes(graph)
UpdateEdgeLengths(graph)

print("Removed all small edges inside the volume/ near the root.")
# print("Removed skeleton voxels corresponding to small edges inside the volume/ near the root.")

Removed all small edges inside the volume/ near the root.


In [97]:
#21-4 Automatically find the root:

leaf_nodes = [node for node, degree in graph.degree if degree == 1]
MaxDistance = -10
root = []
for node in leaf_nodes:
    if distance_transform[node]>MaxDistance:
        root = node
        MaxDistance = distance_transform[node]

print("Automatically found root:", root)

Automatically found root: (440, 222, 359)


In [98]:
#21-5 Visualize: Verify the root is found correct:

# size=[6 if node in leaf_nodes else 2 for node in graph.nodes()],
# color=['black' if node in leaf_nodes else 'grey' for node in graph.nodes()], # 'grey',
# line=dict(color='rgba(255, 0, 0, 0.6)' if SpecialCondition else 'rgba(128, 128, 128, 0.6)',
#             width=6 if SpecialCondition else 2),

fig = mlab.figure(size=(2500, 1400), bgcolor=(1, 1, 1))

x, y, z = zip(*graph.nodes)
mlab.points3d(x, y, z,
              scale_factor=2,
              color=(0.2, 0.2, 0.2), # ['red' if node == root else 'grey' for node in graph.nodes()]
              opacity=0.7) # color=(1, 0, 0)

#  Add the new node with a different scale_factor and color
mlab.points3d(root[0], root[1], root[2],
              scale_factor=14,  # Adjust the scale factor as needed
              color=(1, 0, 0),  # Change color as desired (e.g., red)
              opacity=0.7)

# Create lines for edges
# for edge in graph.edges:
#     src, tgt = edge
#     x = [src[0], tgt[0]]
#     y = [src[1], tgt[1]]
#     z = [src[2], tgt[2]]
#     mlab.plot3d(x, y, z, tube_radius=0.2,
#                 color=(1, 0, 0) if edge in SmallInsideEdges else (0.2, 0.2, 0.2),#(0.2, 0.2, 0.2),
#                 opacity=0.7)

# Create contour for the 3D object
mlab.contour3d(atree, contours=[0.5], opacity=0.2, color=(0, 151/255, 206/255))

winsound.Beep(frequency, duration)
mlab.show()

In [99]:
##23-2 Only test: traverse_edge_and_average for two points:

# index1 = (423, 232, 374)
# index2 = (412, 245, 380)

# skeletonn = copy.deepcopy(skeleton)
# avgdiam, visitedvoxels = traverse_edge_and_average(skeletonn, distance_transform, index1, index2, 5)
# del skeletonn

# if avgdiam is not None:
#     print(f"Average Diameter between {index1} and {index2}: {avgdiam:.2f}")
# else:
#     print("No skeleton found between the specified indices.")

In [100]:
##23-3 Test: Use mayavi to visualize the intended start and end points, plus the taken route, from the above test cell:

# import mayavi.mlab as mlab
# import numpy as np

# # Create a figure
# fig = mlab.figure(size=(800, 800), bgcolor=(1, 1, 1))

# # Plot the skeleton in blue (0 values will be transparent)
# mlab.contour3d(skeleton, color=(0, 0, 1), opacity=0.4)

# # Highlight the visited voxels in red
# for voxel in visitedvoxels:
#     # print(voxel)
#     x, y, z = voxel
#     mlab.points3d(x, y, z, color=(1, 0, 0), scale_factor=1.0) # , mode='sphere'

# for voxel in [index1, index2]:
#     # print(voxel)
#     x, y, z = voxel
#     mlab.points3d(x, y, z, color=(1, 0, 0), scale_factor=4.0) # , mode='sphere'

# # Customize the view and axes as needed
# mlab.view(azimuth=45, elevation=45, distance=20)
# # mlab.axes()

# # Show the visualization
# mlab.show()

In [ ]:
#24 Main function calling the traverse function: $$$ root = () or Enter !!**
# input_string = '(480, 196, 396)'

# Fucntion: fully traverse the directed tree visiting all edges:
def traverse_full_tree(digraph, current_node):
    global EdgeNum
    # Find and process the outgoing edges from the current node
    for successor in digraph.successors(current_node):
        # Run the FindDiameter function for the current edge
        digraph[current_node][successor]['AvgDiameter'] = FindDiameter(current_node, successor, EdgeNum)
        EdgeNum += 1
        # Recursively traverse the tree from the successor node
        traverse_full_tree(digraph, successor)

def FindDiameter(node1, node2, EdgeNum):
    # print(f"Finding avg diameter btw nodes {node1} and {node2}")
    skeletonn = copy.deepcopy(skeleton)
    avgdiam, visitedvoxels = traverse_edge_and_average(skeletonn, distance_transform, node1, node2, EdgeNum)
    del skeletonn
    return avgdiam

# Main: Start the traversal from a specified root node:
import time

winsound.Beep(frequency, duration)
user_input = input("If root was OK just press Enter; if not, enter the root: eg (100, 20, 18): [Cell-24>>Cell-16]: ")
if user_input.strip() == "": # (436, 69, 207) for Case-53! # (480, 196 , 396) for Case-79
    pass
else:
    root = eval(user_input)

digraph = nx.dfs_tree(graph, source=root) # Create a directed graph starting from the specified root node
StartTime = time.time()
EdgeNum = 1
traverse_full_tree(digraph, root)
print('\nDone.')

# Example Error: Attempt 60: Error! Couldn't get from (477, 282, 383) to (466, 270, 425)! Returned None.

In [ ]:
#25 Get the first three edges and their properties as a reality check:

print("Reality check: First 3 edges:\n")
first_three_edges = list(digraph.edges(data=True))[:3]
# Iterate through the first three edges and print their properties
for edge in first_three_edges:
    source, target, attributes = edge
    print(f"Edge ({source} -> {target}):")
    for key, value in attributes.items():
        print(f"  {key}: {value}")
    print("")

In [ ]:
#26 Strahler Analysis: Function+Call: Using DiGraphs: *

import networkx as nx

def StrahlerAnalysis(G): # G: The Tree DiGraph. Returns the edges.
    
    # Find the terminal edges (edges with no successors)
    terminal_edges = [edge for edge in G.edges() if not any(G.successors(edge[1]))]
    # Initialize the order for terminal edges as 1
    for edge in G.edges():
        if edge in terminal_edges: G[edge[0]][edge[1]]['order'] = 1
        else: G[edge[0]][edge[1]]['order'] = 0

    # Perform Strahler analysis starting from terminal edges
    while True:

        # Find all edges with the same order
        edges_by_order = {}
        for edge in G.edges():
            order = G[edge[0]][edge[1]].get('order', 0)
            if order not in edges_by_order:
                edges_by_order[order] = []
            edges_by_order[order].append(edge)

        # If no more order 0, break
        if 0 not in edges_by_order: break

        # Only process edges with order = 0
        for edge in edges_by_order[0]:
            successors = list(G.successors(edge[1]))
            SuccessorOrders = [G[edge[1]][successor]['order'] for successor in successors]
            if not all(SuccessorOrders): continue # If any successor order is 0, skip to next edge
            if len(successors) == 1: # If only one successor, inherit its order
                G[edge[0]][edge[1]]['order'] = G[edge[1]][successors[0]]['order']
            else: # So it's a bifurcation
                MaxOrder = max(SuccessorOrders)
                BinaryComparison = [int(ordr == MaxOrder) for ordr in SuccessorOrders]
                # If more than 2 of the max then increment order
                if sum(BinaryComparison)>=2: NextOrder = MaxOrder + 1
                else: NextOrder = MaxOrder
                G[edge[0]][edge[1]]['order'] = NextOrder

    return G.edges(data=True)

# Main:
edgesresult = StrahlerAnalysis(digraph)
for edge in edgesresult:
    print(f"Edge: {edge[0]} -> {edge[1]}, Order: {edge[2]['order']}")

In [ ]:
## 27 Plot-4 the DiGraph in 3D:

# import networkx as nx
# import plotly.graph_objects as go

# G = digraph

# # Extract node positions and order properties
# node_positions = {node: node for node in G.nodes()}
# edge_orders = nx.get_edge_attributes(G, 'order')

# # Create a trace for nodes
# node_trace = go.Scatter3d(
#     x=[coord[0] for coord in node_positions],
#     y=[coord[1] for coord in node_positions],
#     z=[coord[2] for coord in node_positions],
#     mode='markers',
#     marker=dict(size=2, color='red'),
#     text=[],
#     hoverinfo='text'
# )

# # Create a list of edge traces
# edge_traces = []
# for edge, order in edge_orders.items():
#     source, target = edge
#     x1, y1, z1 = source
#     x2, y2, z2 = target
#     edge_traces.append(go.Scatter3d(
#         x=[x1, x2],
#         y=[y1, y2],
#         z=[z1, z2],
#         mode='lines+text',
#         line=dict(width=4, color='grey'),
#         text=f'{order}',
#         hoverinfo='none',  # Disables hover info for edges
#         textposition='top left'  # Position for the order text
#     ))

# # Create the 3D layout
# layout = go.Layout(
#     scene=dict(
#         xaxis=dict(title='X'),
#         yaxis=dict(title='Y'),
#         zaxis=dict(title='Z'),
#     ),
#     margin=dict(l=0, r=0, b=0, t=0),
# )

# # Create the figure and add traces
# fig = go.Figure(data=edge_traces + [node_trace], layout=layout)

# # Show the interactive plot
# fig.show()

In [ ]:
#28 Statistics of edge orders and counts:

from collections import defaultdict

# Create a dictionary to count edges for each 'order'
order_counts = defaultdict(int)
# Count edges
for u, v, data in digraph.edges(data=True):
    order = data.get("order", 0)
    order_counts[order] += 1
# Print the counts for each 'order' with keys sorted
for order in sorted(order_counts.keys()):
    count = order_counts[order]
    print(f"Order {order}: {count:3} edges")

In [77]:
#29 Add EdgeLength property to digraph edges:

for edge in digraph.edges:
    node1, node2 = edge
    x1, y1, z1 = node1
    x2, y2, z2 = node2
    EdgeLength = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    digraph.edges[edge]['EdgeLength'] = EdgeLength

In [78]:
#30 Save the graph and its properties to a file

nx.write_graphml(digraph, GraphArtifactsFolderName + f"/Case-{CaseNumber}-DiGraph.graphml")

# ## Save all workspace variables as well

# # import pickle
# workspace_variables = globals()
# file_path = "workspace_variables.pkl"
# with open(file_path, 'wb') as file:
#     pickle.dump(workspace_variables, file)

# with open(file_path, 'rb') as file:
#     loaded_workspace_variables = pickle.load(file)

# # Update the global namespace with the loaded variables
# globals().update(loaded_workspace_variables)

In [ ]:
#31 Final step: find edges with target order, calculate mean, and add results to excel file

import openpyxl
from openpyxl.styles import Font, Alignment

font = Font(name='Courier New', size=11)
alignment = Alignment(horizontal='center', vertical='center')

# Find those edges
OrdersPresent = order_counts.keys()
Methods = {1 : [max(OrdersPresent)  , max(OrdersPresent)-2       , '[MX>MX-2]'],
           2 : [max(OrdersPresent)  , max(OrdersPresent)-4       , '[MX>MX-4]'],
           3 : [max(OrdersPresent)-1, max(OrdersPresent)-3       , '[MX-1>MX-3]'],
           4 : [max(OrdersPresent)-1, max(1,max(OrdersPresent)-5), '[MX-1>MX-5|]'],
           5 : [max(OrdersPresent)-2, min(OrdersPresent)         , '[MX-2>MN]'],
           6 : [max(OrdersPresent)-2, min(OrdersPresent)+1       , '[MX-2>MN+1]'],
           7 : [max(OrdersPresent)  , min(OrdersPresent)         , '[MX>MN]'],
           8 : [min(OrdersPresent)  , min(OrdersPresent)         , '[MN>MN]'],
           9 : [min(OrdersPresent)+1, min(OrdersPresent)         , '[MN+1>MN]'],
           10: [min(OrdersPresent)+2, min(OrdersPresent)         , '[MN+2>MN]'],
           11: [min(OrdersPresent)+3, min(OrdersPresent)+1       , '[MN+3>MN+1]']}

# LowerOrder, HigherOrder = 4, 6
# HigherOrder = max(order_counts.keys())
# LowerOrder  = HigherOrder - 2 # -4

excel_file_path = "DiameterResults.xlsx" # Open the existing Excel file
workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active # Select the first sheet
last_row = sheet.max_row # Find the last row in the first column (assuming the first column contains data)
new_row = last_row + 1 # Create a new row

for MethodNum, MthdList in Methods.items():

    HigherOrder = MthdList[0]
    LowerOrder  = MthdList[1]
    selected_edges = [(u, v, data) for u, v, data in digraph.edges(data=True) if LowerOrder <= data.get("order", 0) <= HigherOrder]

    # Print stats
    # print(f"Edges with order in range [{LowerOrder}, {HigherOrder}]:\n")
    # for indx, (u, v, data) in enumerate(selected_edges):
    #     print(f"SelectedEdge-{indx+1:2}: {u} -> {v}; AvgDiameter = {data['AvgDiameter']:12.9f}; Order = {data['order']}; EdgeLength = {data['EdgeLength']:10.6f}")

    # Extract the 'AvgDiameter' values from the selected edges
    avg_diameters = [data["AvgDiameter"] for u, v, data in selected_edges]

    # Calculate the geometric mean of 'AvgDiameter'
    geometric_mean   = np.prod(avg_diameters) ** (1 / len(avg_diameters))
    arithmetic_mean  = sum(avg_diameters) / len(avg_diameters)
    geometric_mean  *= 2*0.38/10 # diameter - mm - cm
    arithmetic_mean *= 2*0.38/10 # diameter - mm - cm

    # print(f"Geometric Mean of AvgDiameters: {geometric_mean} cm")
    # print(f"Arithmetic Mean of AvgDiameters: {arithmetic_mean} cm")

    # Insert data into the new row
    cell = sheet.cell(row=1      , column=1            , value='Case#'       ); cell.font = font; cell.alignment = alignment
    cell = sheet.cell(row=new_row, column=1            , value=CaseNumber    ); cell.font = font; cell.alignment = alignment
    cell = sheet.cell(row=1      , column=MethodNum*2  , value='GeomMean-'+MthdList[2]); cell.font = font; cell.alignment = alignment
    cell = sheet.cell(row=new_row, column=MethodNum*2  , value=geometric_mean); cell.font = font; cell.alignment = alignment
    cell = sheet.cell(row=1      , column=MethodNum*2+1, value='ArthMean-'+MthdList[2]); cell.font = font; cell.alignment = alignment
    cell = sheet.cell(row=new_row, column=MethodNum*2+1, value=arithmetic_mean); cell.font = font; cell.alignment = alignment

workbook.save(excel_file_path) # Save the Excel file
workbook.close() # Close the Excel file

winsound.Beep(1000, 3000)
print(f"Case#{CaseNumber}: Indices written to Excel sheet. All Done!")

In [ ]:
#32 Notes to address:

# Case#58 has a cropped/cut MPA so that lowers the diameters
# SmallInsideEdges are not removed in the first pass! [Next time they were!!!!][?!!] * Takes it twice to fully remove .. [Eg Case-18]
# Case-18 has lots of nuisant small deep branches
# Case-88: Not a tree [and a mess, both with clusters, and final edges]
# Case-19: Obviously more tortuous than normal **
# Case-11: 1 node isolated outside; not a tree
# Case-6: MPA cut too short
# Case-46: Seems like a smaller vessel/MPA-root, what does the dys say?!
# Case-39: 7K cx-cycles!
# Case-22:
# Case-21: MPA OK but much shorter than others
# Should also save the orders list in the excel file
# Case-84: MPA has 20 nuisant edges .. impossible ..
# Any uncircular MPA shape will lead to nuisant edges and then failure *** Including Case-50
# Case-39: has 7K cx-cycles
# Case-44: has 3K cx-cycles

In [88]:
#33 Final visualization: [from cell-21-1]

UpdateEdgeLengths(graph)

fig = mlab.figure(size=(2500, 1400), bgcolor=(1, 1, 1))

x, y, z = zip(*graph.nodes)
mlab.points3d(x, y, z, scale_factor=2, color=(0.2, 0.2, 0.2), opacity=0.7) # color=(1, 0, 0)

# Create lines for edges
for edge in graph.edges:
    src, tgt = edge
    x = [src[0], tgt[0]]
    y = [src[1], tgt[1]]
    z = [src[2], tgt[2]]
    mlab.plot3d(x, y, z, tube_radius=0.2,
                color=(1, 0, 0) if edge in SmallInsideEdges else (0.2, 0.2, 0.2), #(0.2, 0.2, 0.2),
                opacity=0.4)

# Create contour for the 3D object
mlab.contour3d(atree   , contours=[0.5], opacity=0.2, color=(0, 151/255, 206/255))

mlab.contour3d(skeleton, contours=[0.5], opacity=0.4, color=(1, 0, 0))

winsound.Beep(frequency, duration)
mlab.show()

In [ ]:
#34 Partition into Segments
# Partition the 3D A-Tree segment into fragments corresponding to edges:

print(list(digraph.edges)[0])
print(root)

In [ ]:
### shortest_distance_to_skeleton: Original: Inefficient: Won't work

def shortest_distance_to_skeleton(atree, skeleton):
    skeleton_coords = np.argwhere(skeleton == 1)
    output_array = np.zeros_like(atree, dtype=np.float32)

    for x in range(atree.shape[0]):
        for y in range(atree.shape[1]):
            for z in range(atree.shape[2]):
                if atree[x, y, z] == 1:
                    min_dist = float('inf')
                    for skel_x, skel_y, skel_z in skeleton_coords:
                        dist = np.sqrt((x - skel_x)**2 + (y - skel_y)**2 + (z - skel_z)**2)
                        if dist < min_dist:
                            min_dist = dist
                    output_array[x, y, z] = min_dist

    return output_array

output_array = shortest_distance_to_skeleton(atree, skeleton)
print(output_array)

In [ ]:
### Updated Distance Transfrom: Inefficient: Won't work
# Changing the two variables [distnce_trans_2 & nearest_skeleton_coord] only on A-Tree:

atreecoords = np.argwhere(atree == 1)
skeleton_coords = np.argwhere(skeleton == 1)
distnce_trans_2 = np.zeros_like(atree, dtype=np.float32)
nearest_skeleton_coord = np.zeros((*np.shape(atree),3)) # Add a last dimension of 3

for indx, (x, y, z) in enumerate(atreecoords):
    if (indx % 200) == 0: print(indx)
    min_dist = float('inf')
    for skel_x, skel_y, skel_z in skeleton_coords:
        dist = np.sqrt((x - skel_x)**2 + (y - skel_y)**2 + (z - skel_z)**2)
        if dist < min_dist:
            min_dist = dist
            nearest_coords = skel_x, skel_y, skel_z
    distnce_trans_2[x, y, z] = min_dist
    nearest_skeleton_coord[x, y, z] = nearest_coords

In [25]:
import numpy as np
skeletoncoords = np.argwhere(skeleton == 1)

In [24]:
atreegrwn, metadata = nrrd.read("GraphOutputSoheil\Case-46-Final2Label-Atree15mmGrown.nrrd")

In [26]:
import copy
atreegrwnhollow = copy.deepcopy(atreegrwn)

In [27]:
for skcoord in skeletoncoords:
    atreegrwnhollow[*skcoord] = 0
# atreegrwn[skeletoncoords]
# np.shape(skeletoncoords)
# atreegrwn[*skeletoncoords[0:2]]

In [55]:
# Don't rerun .. Compute/Fetch Distance Transform

from scipy.ndimage import distance_transform_edt
EdtPathName = GraphArtifactsFolderName + f"/T-Case-{CaseNumber}-EDT-15mmGrown.nrrd"

# Compute the distance transform
print("Performing the EDT+Indices ..")
# (, distances=None, indices=None)
manipulated_distance_transform, indices = distance_transform_edt(atreegrwnhollow, return_indices=True)

# Or read it in
# distance_transform, _ = nrrd.read(EdtPathName)

Doing the EDT ..


In [59]:
MaskedEdt = manipulated_distance_transform * atree
MaskedIndices = indices * np.stack((atree, atree, atree), axis=0)

In [ ]:
### Saving .. Skip! Takes more than 10 minutes

print("Writing EDT to file ..")
EdtPathName = GraphArtifactsFolderName + f"/T-Case-{CaseNumber}-EDT-15mmGrown-Hollow.nrrd"
nrrd.write(EdtPathName, manipulated_distance_transform)
print("Writing Indices to file ..")
EdtPathName = GraphArtifactsFolderName + f"/T-Case-{CaseNumber}-EDT-15mmGrown-Hollow-Indices.nrrd"
nrrd.write(EdtPathName, indices)

In [54]:
EdtPathName2 = GraphArtifactsFolderName + f"/T-Case-{CaseNumber}-EDT-15mmGrown-Masked.nrrd"
nrrd.write(EdtPathName2, MaskedEdt)

In [65]:
root = (445, 220, 354)
MaskedIndices[:,*root]

array([443, 217, 353])

In [68]:
atreecoords = np.argwhere(atree == 1)
atreepartitioned = copy.deepcopy(atree)

for x, y, z in atreecoords:
    DestinationVoxel = MaskedIndices[: ,x, y, z]
    atreepartitioned[x, y, z] = skeleton[*DestinationVoxel]*3

In [69]:
PathName = GraphArtifactsFolderName + f"/S-Case-{CaseNumber}-Atree-Partitioned.nrrd"
nrrd.write(PathName, atreepartitioned)

In [ ]:
# Experiments-Ver3

# for skcoord in skeletoncoords:
#     print(atreegrwnhollow[*skcoord])
#     print(atreegrwn      [*skcoord])

# atreecoords = np.argwhere(atree == 1)
# np.shape(atreecoords) : (1334406, 3)

# skeleton_coords = np.argwhere(skeleton == 1)
# np.shape(skeleton_coords) : (8130, 3)